# Reorganizing the Data

In [ ]:
import pandas as pd
import os

servers = ["Server_1", "Server_2", "Server_3", "Server_4", "Server_5", "Server_6"]

days = ["Day_1", "Day_2", "Day_3", "Day_4", "Day_5", "Day_6", "Day_7", "Day_8", "Day_9", "Day_10", "Day_11", "Day_12", "Day_13", "Day_14"]

columns = ["node_load1",
           "node_memory_MemAvailable_bytes",
           "node_network_transmit_bytes_total{device=\"eno1\"}"]

for server in servers:
    for day in days:
        if not os.path.exists(f"{server}_Training_Sets/{day}.xlsx"):
            server_metrics_df = pd.read_csv(f"Training_Sets/Training_Set_{day}/physical_level/{server.lower()}.csv", usecols=columns, sep=";")

            for column in columns:
                server_metrics_df[column] = pd.to_numeric(server_metrics_df[column], errors='coerce')

            server_metrics_df.to_excel(f"{server}_Training_Sets/{day}.xlsx", index=False)

In [3]:
import pandas as pd

server_metrics_df = pd.read_excel(f"Server_1_Training_Sets/Day_1.xlsx")

In [ ]:
len(server_metrics_df.columns)

# Dataset Formation

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from tqdm import tqdm

def random_sample_selection(time_series_df, num_samples):
    """
    Seleciona aleatoriamente 'num_samples' amostras do DataFrame.
    
    Args:
        time_series_df (pd.DataFrame): DataFrame com features e target.
        num_samples (int): Número de amostras desejadas.

    Returns:
        pd.DataFrame: DataFrame contendo as amostras selecionadas.
    """
    # Remover linhas com NaNs para evitar problemas
    #time_series_df = time_series_df.dropna()

    # Garantir que o número de amostras não exceda o total disponível
    num_samples_to_select = min(num_samples, len(time_series_df))

    # Selecionar amostras aleatórias
    selected_df = time_series_df.sample(n=num_samples_to_select, random_state=42)

    print(f"Amostras selecionadas aleatoriamente: {num_samples_to_select}")
    return selected_df

In [6]:
server_metrics_df = pd.read_excel(f"Server_1_Training_Sets/Day_14.xlsx")["node_load_1"]

In [ ]:
# Comprimento total do dataset
dataset_length = len(server_metrics_df)

# Pegar as 40 primeiras amostras para servir como features
X = server_metrics_df[0:40]

# Pegar a amostra na posição 40 + 400 para servir como label
y = server_metrics_df[40 + 400]

# Criar um dataframe com base nesta amostra
time_series_df = pd.DataFrame(columns=[f"X{idx+1}" for idx in range(40)] + ["y"])

time_series_df.loc[0] = list(X) + [y]
time_series_df
# Os valores de X1 a X40 estão corretos e y é tomado em relação à X40, ou seja, está 440 amostras a frente de X40

In [ ]:
from tqdm import tqdm

# Carrega o server_metrics com a quantidade de amostras desejada
server_metrics_df = pd.read_excel(f"Server_1_Training_Sets/Day_14.xlsx", nrows=10000)["node_load1"]

# Comprimento total do dataset
dataset_length = len(server_metrics_df)

# Criar um dataframe com base nesta amostra
time_series_df = pd.DataFrame(columns=[f"X{idx+1}" for idx in range(40)] + ["y"])

for idx in tqdm(range(dataset_length - (40 + 400))):
    # Pegar as 40 primeiras amostras para servir como features
    X = server_metrics_df[idx:(40 + idx)]

    # Pegar a amostra na posição 40 + 400 para servir como label
    y = server_metrics_df[(40 + idx) + 400 - 1] # O -1 é inserido devido ao index para ser compatível com o início no index 0

    # Atribuindo o X e o y formados para o dataset
    time_series_df.loc[idx] = list(X) + [y]

In [ ]:
# No laço abaixo, o primeiro (40 + 400) está relacionado com o dataset original. Já o segundo é inserido para que o novo dataset time_series não sofra erro na comparação com X1.
for idx in tqdm(range(0, dataset_length - (40 + 400) - (40 + 400))):
    assert time_series_df["y"][idx] == time_series_df["X1"][(40 + idx - 1) + 400]

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
import os

def time_series_formation(dataset, lookback, step):
    # Comprimento total do dataset
    dataset_length = len(dataset)

    # Criar um dataframe com base nesta amostra
    time_series_df = pd.DataFrame(columns=[f"X{idx+1}" for idx in range(40)] + ["y"])

    for idx in tqdm(range(dataset_length - (lookback + step))):
        # Pegar as 40 primeiras amostras para servir como features
        X = dataset[idx:(lookback + idx)]

        # Pegar a amostra na posição 40 + 400 para servir como label
        y = dataset[(lookback + idx) + step - 1] # O -1 é inserido devido ao index para ser compatível com o início no index 0

        # Atribuindo o X e o y formados para o dataset
        time_series_df.loc[idx] = list(X) + [y]

    # No laço abaixo, o primeiro (40 + 400) está relacionado com o dataset original. Já o segundo é inserido para que o novo dataset time_series não sofra erro na comparação com X1.
    for idx in tqdm(range(0, dataset_length - (lookback + step) - (lookback + step))):
        assert time_series_df["y"][idx] == time_series_df["X1"][(lookback + idx - 1) + step]
    
    print("[TIME SERIES] O Dataset foi adequadamente formado!")

    return time_series_df

def create_and_organize_datasets(pipeline, dataset_mode, dataset_folder):
    os.makedirs(f"Server_1_Training_Sets/{dataset_folder}", exist_ok = True)

    if dataset_mode == "TRAIN":
        days = ["Day_1", "Day_2", "Day_3", "Day_4", "Day_5", "Day_6", "Day_7", "Day_8", "Day_9", "Day_10", "Day_11", "Day_12", "Day_13"]

    elif dataset_mode == "TEST":
        days = ["Day_14"]

    global_representative_df = pd.DataFrame()

    for day in days:
        print(f"\nProcessando {day}...")

        #server_metrics_df = pd.read_excel(f"Server_1_Training_Sets/{day}.xlsx")
        server_metrics_df = pd.read_excel(f"Server_1_Training_Sets/{day}.xlsx", nrows=10000)

        step = 400  # Previsão 400 amostras à frente
        lookback = 40

        # CPU
        if pipeline == "CPU":
            dataset = np.array(server_metrics_df["node_load1"])
            if dataset_mode == "TRAIN":
                file_save_path = f"Server_1_Training_Sets/{dataset_folder}/node_load1_1-13.xlsx"
            elif dataset_mode == "TEST":
                file_save_path = f"Server_1_Training_Sets/{dataset_folder}/node_load1_14.xlsx"

        # RAM
        if pipeline == "RAM":
            dataset = np.array(server_metrics_df["node_memory_MemAvailable_bytes"])
            if dataset_mode == "TRAIN":
                file_save_path = f"Server_1_Training_Sets/{dataset_folder}/node_memory_MemAvailable_bytes_1-13.xlsx"
            elif dataset_mode == "TEST":
                file_save_path = f"Server_1_Training_Sets/{dataset_folder}/node_memory_MemAvailable_bytes_14.xlsx"
        
        # Rede
        # No caso do atributo de rede é necessário derivar.
        if pipeline == "NETWORK":
            dataset = np.array(server_metrics_df["node_network_transmit_bytes_total{device=\"eno1\"}"])
            dataset = pd.Series([dataset[idx + 1] - dataset[idx] for idx in range(len(dataset) - 1)])
            dataset = dataset.drop_duplicates()
            dataset = dataset[dataset < dataset.mean() + 4 * dataset.std()]
            dataset = dataset.reset_index(drop=True)
            if dataset_mode == "TRAIN":
                file_save_path = f"Server_1_Training_Sets/{dataset_folder}/node_network_transmit_bytes_total_1-13.xlsx"
            elif dataset_mode == "TEST":
                file_save_path = f"Server_1_Training_Sets/{dataset_folder}/node_network_transmit_bytes_total_14.xlsx"

        # Construir as features (X) e o target (y)
        time_series_df = time_series_formation(dataset, lookback, step)

        # Tratamento de NaNs
        time_series_df = time_series_df.dropna()

        # Chamar a função de seleção aleatória
        if dataset_mode == "TRAIN":
            #representative_df = random_sample_selection(time_series_df, 10000)
            representative_df = time_series_df[:10000]

        elif dataset_mode == "TEST":
            representative_df = time_series_df.iloc[:10000]

        global_representative_df = pd.concat([global_representative_df, representative_df], ignore_index=True)

    # Salvar dataset
    global_representative_df.to_excel(file_save_path, index=False)

In [ ]:
pipelines = ["CPU"]
dataset_modes = ["TRAIN", "TEST"]

for pipeline in pipelines:
    for dataset_mode in dataset_modes:
        create_and_organize_datasets(pipeline=pipeline, dataset_mode=dataset_mode, dataset_folder="datasets")

# Dataset Treatment

In [70]:
import json
import os

def normalize_datasets(pipeline, dataset_mode, dataset_folder):
    normalization_folder = f"Server_1_Training_Sets/{dataset_folder}/normalization"
    os.makedirs(normalization_folder, exist_ok=True)

    if pipeline == "CPU":
        if dataset_mode == "TRAIN":
            dataset_path = f"Server_1_Training_Sets/{dataset_folder}/node_load1_1-13.xlsx"
            preprocessed_dataset_path = f"Server_1_Training_Sets/{dataset_folder}/cpu_usage_1-13.xlsx"
        elif dataset_mode == "TEST":
            dataset_path = f"Server_1_Training_Sets/{dataset_folder}/node_load1_14.xlsx"
            preprocessed_dataset_path = f"Server_1_Training_Sets/{dataset_folder}/cpu_usage_14.xlsx"
        json_path = f"Server_1_Training_Sets/{dataset_folder}/normalization/max_cpu.json"

    if pipeline == "RAM":
        if dataset_mode == "TRAIN":
            dataset_path = f"Server_1_Training_Sets/{dataset_folder}/node_memory_MemAvailable_bytes_1-13.xlsx"
            preprocessed_dataset_path = f"Server_1_Training_Sets/{dataset_folder}/ram_usage_1-13.xlsx"
        elif dataset_mode == "TEST":
            dataset_path = f"Server_1_Training_Sets/{dataset_folder}/node_memory_MemAvailable_bytes_14.xlsx"
            preprocessed_dataset_path = f"Server_1_Training_Sets/{dataset_folder}/ram_usage_14.xlsx"
        json_path = f"Server_1_Training_Sets/{dataset_folder}/normalization/max_ram.json"

    if pipeline == "NETWORK":
        if dataset_mode == "TRAIN":
            dataset_path = f"Server_1_Training_Sets/{dataset_folder}/node_network_transmit_bytes_total_1-13.xlsx"
            preprocessed_dataset_path = f"Server_1_Training_Sets/{dataset_folder}/network_usage_1-13.xlsx"
        elif dataset_mode == "TEST":
            dataset_path = f"Server_1_Training_Sets/{dataset_folder}/node_network_transmit_bytes_total_14.xlsx"
            preprocessed_dataset_path = f"Server_1_Training_Sets/{dataset_folder}/network_usage_14.xlsx"
        json_path = f"Server_1_Training_Sets/{dataset_folder}/normalization/max_network.json"

    if not os.path.exists(preprocessed_dataset_path):

        print(f"[Pipeline: {pipeline} | Dataset Mode: {dataset_mode}] Normalização iniciada")

        # Carregando o dataset
        dataset = pd.read_excel(dataset_path)

        if dataset_mode == "TRAIN":

            # Inicializar dicionários para armazenar os valores máximos de cada coluna dentro de 3 desvios padrões
            max_values = {}

            # Iterar sobre as colunas de features X1 até X40
            for col in [f"X{i}" for i in range(1, 41)]:
                # Calcular a média e o desvio padrão para a coluna atual
                mean_col = dataset[col].mean()
                std_col = dataset[col].std()
                
                # Definir os limites de 3 desvios padrões
                lower_limit = mean_col - 3 * std_col
                upper_limit = mean_col + 3 * std_col
                
                # Filtrar os dados dentro dos limites e encontrar o valor máximo
                max_value = dataset[(dataset[col] >= lower_limit) & (dataset[col] <= upper_limit)][col].max()
                max_values[col] = max_value

            # Fazer o mesmo para a coluna 'y'
            mean_y = dataset['y'].mean()
            std_y = dataset['y'].std()
            lower_limit_y = mean_y - 3 * std_y
            upper_limit_y = mean_y + 3 * std_y
            max_y = dataset[(dataset['y'] >= lower_limit_y) & (dataset['y'] <= upper_limit_y)]['y'].max()
            max_values['y'] = max_y

            max_values_converted = {key: int(value) if isinstance(value, np.integer) else float(value) for key, value in max_values.items()}

            # Salvar o dicionário convertido em um arquivo JSON
            with open(json_path, "w") as f:
                json.dump(max_values_converted, f, indent=4)

        # Carregar o JSON como um dicionário
        with open(json_path, "r") as f:
            max_values = json.load(f)

        # Normalizar o dataset usando os valores máximos encontrados
        normalized_dataset = dataset.copy()
        for col in max_values.keys():
            normalized_dataset[col] = normalized_dataset[col] / max_values[col]

        # CPU
        if pipeline == "CPU":
            #...
            pass
            
        # RAM
        if pipeline == "RAM":
            normalized_dataset = 1 - normalized_dataset
            normalized_dataset[normalized_dataset < 0] = 0

        # Network
        if pipeline == "NETWORK":
            normalized_dataset[normalized_dataset < 0] = 0

        print(f"[Pipeline: {pipeline} | Dataset Mode: {dataset_mode}] Normalização finalizada")

        normalized_dataset.to_excel(preprocessed_dataset_path, index=False)

        print(f"[Pipeline: {pipeline} | Dataset Mode: {dataset_mode}] Conjunto de dados normalizado salvo")

    else:
        print(f"[Pipeline: {pipeline} | Dataset Mode: {dataset_mode}] O dataset já foi construído")

In [ ]:
pipelines = ["CPU"]
dataset_modes = ["TRAIN", "TEST"]

for pipeline in pipelines:
    for dataset_mode in dataset_modes:
        normalize_datasets(pipeline=pipeline, dataset_mode=dataset_mode, dataset_folder="datasets")